In [1]:
import sys
sys.path.append('/Users/erikweilandt/Library/Caches/pypoetry/virtualenvs/tepem-FeojsIC2-py3.11/lib/python3.11/site-packages')

In [2]:
from sympy import symbols, simplify, prod, plot, diff
import numpy as np

In [3]:
def LagrangPoly(y,order,i,yi=None):
    if yi==None:
        yi=symbols('y:%d'%(order+1))
    index = list(range(order+1))
    index.pop(i)
    return prod([(y-yi[j])/(yi[i]-yi[j]) for j in index])

reference: https://www.longqi.cf/python/2014/03/24/implement-of-lagrange-polynomial-in-sympy/

In [4]:
order_of_pols = 4
nodes = np.linspace(0,1, order_of_pols+1)

In [5]:
x, y=symbols('x y')
LagrangPoly(y,order_of_pols,0)

(y - y1)*(y - y2)*(y - y3)*(y - y4)/((y0 - y1)*(y0 - y2)*(y0 - y3)*(y0 - y4))

## $\mathbb{P}_d$ shape functions

In [6]:
pols = {}
for i in range(order_of_pols+1):
    pols[f"xi_{i}"] = simplify(LagrangPoly(y,order_of_pols,i,list(nodes)))
pols

{'xi_0': 1.0*(y - 1)*(1.33333333333333*y - 1.0)*(2.0*y - 1.0)*(4.0*y - 1.0),
 'xi_1': y*(-42.6666666666667*y**3 + 96.0*y**2 - 69.3333333333333*y + 16.0),
 'xi_2': 4.0*y*(y - 1)*(4.0*y - 3.0)*(4.0*y - 1.0),
 'xi_3': y*(-42.6666666666667*y**3 + 74.6666666666667*y**2 - 37.3333333333333*y + 5.33333333333333),
 'xi_4': y*(10.6666666666667*y**3 - 16.0*y**2 + 7.33333333333333*y - 1.0)}

In [7]:
#for pol in pols.values():
#    plot(pol, (y,0,1))

### Hardcoded for order 4 polynomials

In [8]:
#phi_0 = simplify(LagrangPoly(y,4,0,[0, 0.25, 0.5, 0.75, 1]))
#phi_1 = simplify(LagrangPoly(y,4,1,[0, 0.25, 0.5, 0.75, 1]))
#phi_2 = simplify(LagrangPoly(y,4,2,[0, 0.25, 0.5, 0.75, 1]))
#phi_3 = simplify(LagrangPoly(y,4,3,[0, 0.25, 0.5, 0.75, 1]))
#phi_4 = simplify(LagrangPoly(y,4,4,[0, 0.25, 0.5, 0.75, 1]))

In [9]:
# plot(phi_0,phi_1,phi_2,phi_3,phi_4, (y,0,1))

## $\mathbb{P}_1$ shape functions

In [10]:
psi_0 = 1 - x
psi_1 = x

In [11]:
# plot(psi_0, psi_1, (x,0,1))

## $\mathbb{Q}_{1,4}$ shape functions

In [12]:
q_sf = {} 
for i, sf in enumerate(pols.values()):
    q_sf[f"phi_{i}"] = psi_0 * sf
for i, sf in enumerate(pols.values(), order_of_pols+1):
    q_sf[f"phi_{i}"] = psi_1 * sf
q_sf

{'phi_0': 1.0*(1 - x)*(y - 1)*(1.33333333333333*y - 1.0)*(2.0*y - 1.0)*(4.0*y - 1.0),
 'phi_1': y*(1 - x)*(-42.6666666666667*y**3 + 96.0*y**2 - 69.3333333333333*y + 16.0),
 'phi_2': 4.0*y*(1 - x)*(y - 1)*(4.0*y - 3.0)*(4.0*y - 1.0),
 'phi_3': y*(1 - x)*(-42.6666666666667*y**3 + 74.6666666666667*y**2 - 37.3333333333333*y + 5.33333333333333),
 'phi_4': y*(1 - x)*(10.6666666666667*y**3 - 16.0*y**2 + 7.33333333333333*y - 1.0),
 'phi_5': 1.0*x*(y - 1)*(1.33333333333333*y - 1.0)*(2.0*y - 1.0)*(4.0*y - 1.0),
 'phi_6': x*y*(-42.6666666666667*y**3 + 96.0*y**2 - 69.3333333333333*y + 16.0),
 'phi_7': 4.0*x*y*(y - 1)*(4.0*y - 3.0)*(4.0*y - 1.0),
 'phi_8': x*y*(-42.6666666666667*y**3 + 74.6666666666667*y**2 - 37.3333333333333*y + 5.33333333333333),
 'phi_9': x*y*(10.6666666666667*y**3 - 16.0*y**2 + 7.33333333333333*y - 1.0)}

In [13]:
grad_q = {}
for i, sf in enumerate(q_sf.values()):
    grad_x = diff(sf, x)
    grad_y = diff(sf, y)
    grad_q[f"grad_phi_{i}"] = (grad_x, grad_y)
grad_q

{'grad_phi_0': (-1.0*(y - 1)*(1.33333333333333*y - 1.0)*(2.0*y - 1.0)*(4.0*y - 1.0),
  4.0*(1 - x)*(y - 1)*(1.33333333333333*y - 1.0)*(2.0*y - 1.0) + 2.0*(1 - x)*(y - 1)*(1.33333333333333*y - 1.0)*(4.0*y - 1.0) + 1.33333333333333*(1 - x)*(y - 1)*(2.0*y - 1.0)*(4.0*y - 1.0) + 1.0*(1 - x)*(1.33333333333333*y - 1.0)*(2.0*y - 1.0)*(4.0*y - 1.0)),
 'grad_phi_1': (-y*(-42.6666666666667*y**3 + 96.0*y**2 - 69.3333333333333*y + 16.0),
  y*(1 - x)*(-128.0*y**2 + 192.0*y - 69.3333333333333) + (1 - x)*(-42.6666666666667*y**3 + 96.0*y**2 - 69.3333333333333*y + 16.0)),
 'grad_phi_2': (-4.0*y*(y - 1)*(4.0*y - 3.0)*(4.0*y - 1.0),
  16.0*y*(1 - x)*(y - 1)*(4.0*y - 3.0) + 16.0*y*(1 - x)*(y - 1)*(4.0*y - 1.0) + 4.0*y*(1 - x)*(4.0*y - 3.0)*(4.0*y - 1.0) + 4.0*(1 - x)*(y - 1)*(4.0*y - 3.0)*(4.0*y - 1.0)),
 'grad_phi_3': (-y*(-42.6666666666667*y**3 + 74.6666666666667*y**2 - 37.3333333333333*y + 5.33333333333333),
  y*(1 - x)*(-128.0*y**2 + 149.333333333333*y - 37.3333333333333) + (1 - x)*(-42.6666666666667*

## Generate shape function module

In [14]:
for sf_name, sf in q_sf.items():
    def_header = f"def {sf_name}(x: float, y: float) -> float:\n"
    body = f"\treturn {sf}"
    print(def_header + body)

def phi_0(x: float, y: float) -> float:
	return 1.0*(1 - x)*(y - 1)*(1.33333333333333*y - 1.0)*(2.0*y - 1.0)*(4.0*y - 1.0)
def phi_1(x: float, y: float) -> float:
	return y*(1 - x)*(-42.6666666666667*y**3 + 96.0*y**2 - 69.3333333333333*y + 16.0)
def phi_2(x: float, y: float) -> float:
	return 4.0*y*(1 - x)*(y - 1)*(4.0*y - 3.0)*(4.0*y - 1.0)
def phi_3(x: float, y: float) -> float:
	return y*(1 - x)*(-42.6666666666667*y**3 + 74.6666666666667*y**2 - 37.3333333333333*y + 5.33333333333333)
def phi_4(x: float, y: float) -> float:
	return y*(1 - x)*(10.6666666666667*y**3 - 16.0*y**2 + 7.33333333333333*y - 1.0)
def phi_5(x: float, y: float) -> float:
	return 1.0*x*(y - 1)*(1.33333333333333*y - 1.0)*(2.0*y - 1.0)*(4.0*y - 1.0)
def phi_6(x: float, y: float) -> float:
	return x*y*(-42.6666666666667*y**3 + 96.0*y**2 - 69.3333333333333*y + 16.0)
def phi_7(x: float, y: float) -> float:
	return 4.0*x*y*(y - 1)*(4.0*y - 3.0)*(4.0*y - 1.0)
def phi_8(x: float, y: float) -> float:
	return x*y*(-42.66666

In [15]:
for sf_name, sf in grad_q.items():
    def_header = f"def {sf_name}(x: float, y: float) -> npt.NDArray[np.float64]:\n"
    body = f"\tphi_dx = {sf[0]}\n\tphi_dy = {sf[1]}\n"
    return_str = f"\treturn np.array([phi_dx, phi_dy])"
    print(def_header + body + return_str)

def grad_phi_0(x: float, y: float) -> npt.NDArray[np.float64]:
	phi_dx = -1.0*(y - 1)*(1.33333333333333*y - 1.0)*(2.0*y - 1.0)*(4.0*y - 1.0)
	phi_dy = 4.0*(1 - x)*(y - 1)*(1.33333333333333*y - 1.0)*(2.0*y - 1.0) + 2.0*(1 - x)*(y - 1)*(1.33333333333333*y - 1.0)*(4.0*y - 1.0) + 1.33333333333333*(1 - x)*(y - 1)*(2.0*y - 1.0)*(4.0*y - 1.0) + 1.0*(1 - x)*(1.33333333333333*y - 1.0)*(2.0*y - 1.0)*(4.0*y - 1.0)
	return np.array([phi_dx, phi_dy])
def grad_phi_1(x: float, y: float) -> npt.NDArray[np.float64]:
	phi_dx = -y*(-42.6666666666667*y**3 + 96.0*y**2 - 69.3333333333333*y + 16.0)
	phi_dy = y*(1 - x)*(-128.0*y**2 + 192.0*y - 69.3333333333333) + (1 - x)*(-42.6666666666667*y**3 + 96.0*y**2 - 69.3333333333333*y + 16.0)
	return np.array([phi_dx, phi_dy])
def grad_phi_2(x: float, y: float) -> npt.NDArray[np.float64]:
	phi_dx = -4.0*y*(y - 1)*(4.0*y - 3.0)*(4.0*y - 1.0)
	phi_dy = 16.0*y*(1 - x)*(y - 1)*(4.0*y - 3.0) + 16.0*y*(1 - x)*(y - 1)*(4.0*y - 1.0) + 4.0*y*(1 - x)*(4.0*y - 3.0)*(4.0*y - 1.0